In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftfreq
from scipy import signal
from scipy.signal import butter, lfilter
from scipy.stats import skew
from scipy.signal import welch
from scipy import stats

In [80]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

rutas_muestras1 = [
    "../PRUEBAS/3 canales/Ruben Dario/1/arriba",
    "../PRUEBAS/3 canales/Ruben Dario/2/arriba",
    "../PRUEBAS/3 canales/Ruben Dario/3/arriba",
    "../PRUEBAS/3 canales/Ruben Dario/4/arriba",
    "../PRUEBAS/3 canales/Ruben Dario/5/arriba",
    "../PRUEBAS/3 canales/Ruben Dario/1/derecha",
    "../PRUEBAS/3 canales/Ruben Dario/2/derecha",
    "../PRUEBAS/3 canales/Ruben Dario/3/derecha",
    "../PRUEBAS/3 canales/Ruben Dario/4/derecha",
    "../PRUEBAS/3 canales/Ruben Dario/5/derecha",
    "../PRUEBAS/3 canales/Ruben Dario/1/abajo",
    "../PRUEBAS/3 canales/Ruben Dario/2/abajo",
    "../PRUEBAS/3 canales/Ruben Dario/3/abajo",
    "../PRUEBAS/3 canales/Ruben Dario/4/abajo",
    "../PRUEBAS/3 canales/Ruben Dario/5/abajo",
    "../PRUEBAS/3 canales/Ruben Dario/1/izquierda",
    "../PRUEBAS/3 canales/Ruben Dario/2/izquierda",
    "../PRUEBAS/3 canales/Ruben Dario/3/izquierda",
    "../PRUEBAS/3 canales/Ruben Dario/4/izquierda",
    "../PRUEBAS/3 canales/Ruben Dario/5/izquierda",
    ]

In [81]:
def calculo_caracte3(sen):

    fs = 250
    lowcut = 2
    highcut = 30

    # Arreglo de características
    caracteristicas = np.empty((3, 9))  # 3 filas (sen('1'), sen('2'), sen('3')), 9 columnas (características)

    for i in range(1, 4):
        # Obtener los datos correspondientes a sen('0')
        sig = butter_bandpass_filter(sen[str(i-1)], lowcut, highcut, fs, order=6)
        
        # Inicializar las listas para almacenar las medidas estadísticas
        mean_values = []
        std_values = []
        energy_values = []
        curtosis = []
        skewness = []
        RMS_Value = []
        ancho_banda = []

        # Calcular las medidas estadísticas
        mean = np.mean(sig)
        mean_values.append(mean)

        std = np.std(sig)
        std_values.append(std)

        energy = np.sum(np.square(sig)) / len(sig)
        energy_values.append(energy)

        curtosis = stats.kurtosis(sig)

        skewness = skew(sig)

        RMS_Value = np.sqrt(np.mean(sig**2))

        cruces_por_cero = np.where(np.diff(np.signbit(sig)))[0]
        contador_cruces = len(cruces_por_cero)

        frecuencias, periodograma = welch(sig)
        media_periodograma = np.mean(periodograma)

        n = len(sig)
        frecuencias = fftfreq(n)
        transformada = fft(sig)
        transformada_abs = np.abs(transformada)
        indice_max = np.argmax(transformada_abs)
        frecuencia_max = frecuencias[indice_max]
        ancho_banda = 2 * np.abs(frecuencia_max)
        
        # Guardar las características en el arreglo
        caracteristicas[i-1] = [mean, std, energy, curtosis, skewness, RMS_Value, contador_cruces, media_periodograma, ancho_banda]

        return caracteristicas

In [82]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.signal import butter, welch
from scipy.fftpack import fft, fftfreq
from scipy.stats import skew
fs = 250
lowcut = 2
highcut = 30
# Crear el DataFrame 'carac' con las columnas correspondientes a las características
carac = pd.DataFrame(columns=['mean', 'std', 'energy', 'curtosis', 'skewness', 'RMS', '#cruce_0', 'media_welch', 'ancho_banda'])

# Recorrer cada ruta de muestra en el ciclo for
for i, ruta_muestra in enumerate(rutas_muestras1):
    for j in range(1, 4):
        # Leer el archivo CSV de la muestra
        sen = pd.read_csv(ruta_muestra, sep=';')

        # Calcular las características
        sig = butter_bandpass_filter(sen[str(j-1)], lowcut, highcut, fs, order=6)
        mean = np.mean(sig)
        std = np.std(sig)
        energy = np.sum(np.square(sig)) / len(sig)
        curtosis = stats.kurtosis(sig)
        skewness = skew(sig)
        RMS_Value = np.sqrt(np.mean(sig**2))
        cruces_por_cero = np.where(np.diff(np.signbit(sig)))[0]
        contador_cruces = len(cruces_por_cero)
        frecuencias, periodograma = welch(sig)
        media_periodograma = np.mean(periodograma)
        n = len(sig)
        frecuencias = fftfreq(n)
        transformada = fft(sig)
        transformada_abs = np.abs(transformada)
        indice_max = np.argmax(transformada_abs)
        frecuencia_max = frecuencias[indice_max]
        ancho_banda = 2 * np.abs(frecuencia_max)

        # Asignar los valores al DataFrame 'carac'
        carac.loc[j-1+i*3, 'mean'] = mean
        carac.loc[j-1+i*3, 'std'] = std
        carac.loc[j-1+i*3, 'energy'] = energy
        carac.loc[j-1+i*3, 'curtosis'] = curtosis
        carac.loc[j-1+i*3, 'skewness'] = skewness
        carac.loc[j-1+i*3, 'RMS'] = RMS_Value
        carac.loc[j-1+i*3, '#cruce_0'] = contador_cruces
        carac.loc[j-1+i*3, 'media_welch'] = media_periodograma
        carac.loc[j-1+i*3, 'ancho_banda'] = ancho_banda

valores = ["arriba"] * 15 + ["derecha"] * 15 + ["abajo"] * 15 + ["izquierda"] * 15

# Agregar la nueva columna al DataFrame
carac['caracteristica'] = valores


In [83]:
carac

,mean,std,energy,curtosis,skewness,RMS,#cruce_0,media_welch,ancho_banda,caracteristica
0,-0.009052,2.02303,4.092733,0.497256,0.105567,2.02305,305,8.642743,0.039315,arriba
1,-0.008064,2.028921,4.116585,0.508128,0.240627,2.028937,288,8.812016,0.039315,arriba
2,-0.009244,2.021956,4.08839,0.479564,0.177387,2.021977,302,8.695842,0.039315,arriba
3,-0.008817,2.396003,5.74091,0.536607,0.015141,2.39602,269,12.044945,0.035282,arriba
4,-0.007437,2.371054,5.62195,0.510936,0.040509,2.371065,289,11.813339,0.022177,arriba
5,-0.005986,2.341536,5.482827,0.393473,0.033762,2.341544,295,11.556464,0.035282,arriba
6,-0.017494,2.269501,5.150942,0.211271,-0.023615,2.269569,283,10.256404,0.037298,arriba
7,-0.017005,2.278428,5.191524,0.194166,-0.033072,2.278492,273,10.390583,0.037298,arriba
8,-0.014796,2.273892,5.170805,0.14064,-0.023258,2.27394,261,10.368884,0.037298,arriba
9,-0.009787,2.392171,5.722576,1.181028,-0.072919,2.392191,295,11.098165,0.034274,arriba


In [78]:
carac.to_csv("caracRuben",index=0,sep=';')